## Implementación modelo de clasificación con árboles de decisión

Realizado por **Sergio Peñuela (201922873)**

### 1. Importar librerías y llevar a cabo la carga de datos

In [64]:
import pandas as pd
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 50)
import numpy as np
np.random.seed(3301)
import pandas as pd
from sklearn.preprocessing import LabelEncoder

from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import plot_confusion_matrix

from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import KFold

import matplotlib as mplt
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import tree
from sklearn.utils import resample
import pickle

# Vectorización de texto
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk

In [65]:
# Carga de datos
df = pd.read_csv("../data/processed_data_min.csv")
df.sample(5)

,class,tokens
21738,0,aaa lit stupidest person ex lik ev comp lik is...
26627,0,sound rain on calm sound ear sound rain one ca...
135198,1,clos amam tir liv tir easy target men tran tir...
140848,0,guess kiss girl meampxbbtw saw post said win s...
79670,0,delet reddit account mean quest delete reddit ...


In [66]:
df.describe()

,class
count,195700.000000
mean,0.437072
std,0.496026
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [67]:
# Contar valores nulos
df.isnull().sum()

class      0
tokens    91
dtype: int64

In [68]:
# Eliminar valores nulos
df.dropna(inplace=True)

### 2. Construcción del modelo y variable objetivo

In [69]:
# Se selecciona la variable objetivo, en este caso "Class".
Y = df['class']
# Del conjunto de datos se elimina la variable "Class"
X = df['tokens']

In [70]:
# Vectorización de texto
vectorizer = TfidfVectorizer(max_df=1200, min_df=2, max_features=10000, ngram_range=(1,2))
X_count = vectorizer.fit_transform(X)

In [71]:
# Dividir los datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X_count, Y, test_size=0.15, random_state=0)

#### Antes de seguir se hará primero un ajuste de hiperparámetros para poder obtener el mejor valor posible.

In [72]:
# Fijemos el número de particiones. Utilizaremos K = 10.
particiones = KFold(n_splits=10, shuffle=True, random_state = 0)

In [73]:
# Establecemos el espacio de búsqueda para los hiperparámetros que deseamos ajustar. 
param_grid = {'criterion':['gini', 'entropy'],'max_depth':[4,6,8,10,20],'min_samples_split':[2, 3, 4, 5]}

In [74]:
# Definimos el modelo sin ningún valor de estos hiperparámetros
arbol = DecisionTreeClassifier(random_state=0)

In [75]:
# Ahora utilizamos GridSearch sobre el grid definido y con 10 particiones en la validación cruzada.
mejor_modelo = GridSearchCV(arbol, param_grid, cv=particiones)
# Ajuste del modelo
mejor_modelo.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=10, random_state=0, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=0),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [4, 6, 8, 10, 20],
                         'min_samples_split': [2, 3, 4, 5]})

In [76]:
# Podemos ver cuál fue el resultado de la búsqueda (mejores valores de hiperparámetros)
mejor_modelo.best_params_

{'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 3}

In [77]:
# Obtener el mejor modelo.
arbol_final = mejor_modelo.best_estimator_
# Probemos ahora este modelo sobre test.
y_pred_train = arbol_final.predict(X_train)
y_pred_test = arbol_final.predict(X_test)
print('Exactitud sobre entrenamiento: %.2f' % accuracy_score(y_train, y_pred_train))
print('Exactitud sobre test: %.2f' % accuracy_score(y_test, y_pred_test))

Exactitud sobre entrenamiento: 0.65
Exactitud sobre test: 0.65


In [78]:
# Generar reporte de clasificación y métricas
print('Resultados:')
print('Exactitud: %.2f' % accuracy_score(y_test, y_pred_test))
print("Recall: {}".format(recall_score(y_test,y_pred_test)))
print("Precisión: {}".format(precision_score(y_test,y_pred_test)))
print("Puntuación F1: {}".format(f1_score(y_test,y_pred_test)))
print(classification_report(y_test, y_pred_test))

Resultados:
Exactitud: 0.65
Recall: 0.205100039231071
Precisión: 0.9670736219015909
Puntuación F1: 0.33842568617296737
              precision    recall  f1-score   support

           0       0.62      0.99      0.76     16597
           1       0.97      0.21      0.34     12745

    accuracy                           0.65     29342
   macro avg       0.79      0.60      0.55     29342
weighted avg       0.77      0.65      0.58     29342

